In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import json
import sys

In [42]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 100.0.4896
[WDM] - Get LATEST chromedriver version for 100.0.4896 google-chrome
[WDM] - Driver [C:\Users\GuoshengCai\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache


In [16]:
def find_department_pages(driver, departments_url, no_pages, css_selector, nth_element=None):
    driver.get(departments_url)
    items = []
    for i in range(no_pages):
        if i != 0:
            to_click = driver.find_element(By.LINK_TEXT, 'Next')
            to_click.click()
        elements = driver.find_elements(By.CSS_SELECTOR, css_selector)
        if nth_element is not None:
            elements = elements[nth_element].find_elements(By.CSS_SELECTOR,"*")
        for element in elements:
            try:
                tmp = element.find_element(By.TAG_NAME, 'a')
                item = {'department': tmp.get_attribute('textContent'), 'url': tmp.get_attribute('href')}
                items.append(item)
            except:
                pass
    return items

In [17]:
def find_faculty_page(driver, url):
    driver.get(url)
    elements = driver.find_elements(By.XPATH, 
    '//a[contains(@href,"faculty") or contains(@href,"profile") or contains(@href,"people") or contains(@href,"directory") or contains(@href,"staff")]')
    if len(elements) == 0:
        lk = None
    else:
        lk = select_lk(elements)
    return lk

In [18]:
def select_lk(elements):
    priority_dict = {'faculty directory': 7, 'faculty list': 6, 'faculty': 5, 'people': 4,'directory': 3,
                     'profiles': 2, 'staff': 1}
    lk = None
    priority = -1
    for element in elements:
        s = element.get_attribute("innerText").lower()
        curr_priority = 0
        for t in priority_dict:
            if t == s:
                curr_priority = priority_dict.get(t)
                break
        if curr_priority > priority:
            priority = curr_priority
            lk = element.get_attribute('href')
    return lk

# Columbia

In [24]:
departments_url = "https://www.columbia.edu/content/academics/departments"
no_pages = 5
css_selector = '.dynamic-grid-listing-item.grid-item.angular-animate.ng-trans.ng-trans-fade-down.ng-scope'
columbia_departments = find_department_pages(driver, departments_url, no_pages, css_selector)

In [91]:
for item in columbia_departments:
    url = item.get('url')
    item.update({'faculty_page': find_faculty_page(driver, url)})

In [93]:
with open(sys.path[0] + '/../data/columbia/columbia_directories_updated.json', 'w') as f:
    json.dump(columbia_departments, f)

# Dartmouth

In [123]:
departments_url = "https://home.dartmouth.edu/academics/departments-programs"
no_pages = 1
nth_element = 2
css_selector = '.clearfix.text-formatted.field.field--name-body.field--type-text-with-summary.field--label-hidden.field__item'
dartmouth_departments = find_department_pages(driver, departments_url, no_pages, css_selector, nth_element)

In [131]:
# https://stackoverflow.com/questions/7867537/how-to-select-a-drop-down-menu-value-with-selenium-using-python
def find_faculty_page_dartmouth(driver, url):
    driver.get(url)
    try:
        element = driver.find_element(By.XPATH, 
        '//a[contains(@href,"people")]') 
        lk = element.get_attribute('href')
        driver.get(lk)
        # https://stackoverflow.com/questions/26958193/how-to-select-any-element-from-the-web-element-with-display-none-attribute-us
        driver.execute_script("document.getElementById('edit-field-people-local-tags-tid').style.display='inline-block';")
        element = WebDriverWait(driver, 60).until(EC.visibility_of_element_located((By.ID, 'edit-field-people-local-tags-tid')))
        select = Select(element)
        try:
            select.select_by_visible_text('Faculty')
        except:
            pass

        try:
            select.select_by_visible_text('faculty')
        except:
            select.select_by_index(1)

        to_click = driver.find_element(By.ID, 'edit-submit-people')
        to_click.click()
        currentURL = driver.current_url
        return currentURL
    except:
        return None

In [132]:
for item in dartmouth_departments:
    url = item.get('url')
    item.update({'faculty_page': find_faculty_page_dartmouth(driver, url)})

In [134]:
with open(sys.path[0] + '/../data/dartmouth/dartmouth_directories.json', 'w') as f:
    json.dump(dartmouth_departments, f)

# Yale

In [26]:
departments_url = "https://www.yale.edu/academics/departments-programs"
no_pages = 1
css_selector = '.department_item_heading'
yale_departments = find_department_pages(driver, departments_url, no_pages, css_selector)

In [31]:
for item in yale_departments:
    url = item.get('url')
    item.update({'faculty_page': find_faculty_page(driver, url)})

In [34]:
with open(sys.path[0] + '/../data/yale/yale_directories.json', 'w') as f:
    json.dump(yale_departments, f)

# Princeton

In [1]:
# only undergraduate departments
def find_department_pages_princeton_undergrad(driver):
    driver.get('https://admission.princeton.edu/academics/degrees-departments')
    elements = driver.find_elements(By.TAG_NAME, "p")
    items = []
    for element in elements:
        #element = element.find_element(By.XPATH, "..")
        try:
            tmp = element.find_element(By.TAG_NAME, 'a')
            item = {'department': tmp.get_attribute('textContent'), 'url': tmp.get_attribute('href')}
            items.append(item)
        except:
            pass
    return items

In [36]:
#princeton_grad = find_department_pages_princeton_undergrad(driver)

In [4]:
princeton_departments = find_department_pages_princeton_undergrad(driver)

In [6]:
princeton_departments = princeton_departments[1:-1]

In [41]:
for item in princeton_departments:
    url = item.get('url')
    item.update({'faculty_page': find_faculty_page(driver, url)})

In [42]:
with open(sys.path[0] + '/../data/princeton/princeton_directories.json', 'w') as f:
    json.dump(princeton_departments, f)

# Brown

In [8]:
departments_url = "https://www.brown.edu/academics/departments"
no_pages = 1
css_selector = '.news-items'
brown_departments = find_department_pages(driver, departments_url, no_pages, css_selector)

In [4]:
def find_faculty_page_brown(driver, url):
    driver.get(url)
    elements = driver.find_elements(By.XPATH, 
    '//a[contains(@href,"faculty") or contains(@href,"profile") or contains(@href,"people") or contains(@href,"directory") or contains(@href,"staff")]')
    if len(elements) == 0:
        lk = None
    else:
        lk = select_lk_brown(elements)
    return lk

In [11]:
def select_lk_brown(elements):
    priority_dict = {'faculty directory': 7, 'faculty list': 6, 'faculty': 5, 'people': 4,'directory': 3,
                     'profiles': 2, 'staff': 1}
    lk = None
    priority = -1
    for element in elements:
        if element.get_attribute('href') in ['https://www.brown.edu/gateway/faculty', 'http://www.brown.edu/gateway/faculty']:
            continue
        s = element.get_attribute("innerText").lower()
        curr_priority = 0
        for t in priority_dict:
            if t == s:
                curr_priority = priority_dict.get(t)
                break
        if curr_priority > priority:
            priority = curr_priority
            lk = element.get_attribute('href')
    return lk

In [12]:
for item in brown_departments:
    url = item.get('url')
    item.update({'faculty_page': find_faculty_page_brown(driver, url)})

In [14]:
with open(sys.path[0] + '/../data/brown/brown_directories.json', 'w') as f:
    json.dump(brown_departments, f)

# Cornell

In [6]:
def find_department_pages_cornell(driver):
    driver.get('https://www.cornell.edu/academics/departments.cfm')
    items = []
    elements = driver.find_elements(By.TAG_NAME, 'li')
    for element in elements:
        try:
            tmp = element.find_element(By.TAG_NAME, 'a')
            item = {'department': tmp.get_attribute('textContent'), 'url': tmp.get_attribute('href')}
            items.append(item)
        except:
            pass
    return items

In [7]:
cornell_departments = find_department_pages_cornell(driver)

In [8]:
for i in range(len(cornell_departments)):
    print(i, cornell_departments[i].get('department'))

0 About Cornell
1 University Facts
2 Maps & Directions
3 Mission & Vision
4 Core Values
5 Locations
6 Ithaca
7 New York City
8 Diversity
9 Indigenous Engagement
10 Economic Impact
11 Sustainability
12 Global
13 Cornell Leadership
14 Admissions
15 Apply
16 Admissions
17 Dates and Deadlines
18 Costs and Aid 
19 Admissions
20 Fields of Study
21 Financial Support
22 Summer Sessions
23 Winter Sessions
24 Part-time Study
25 Academics
26 Fields of Study
27 Colleges & Schools
28 Departments
29 Library
30 Continuing Education
31 Office of the Provost
32 Academic Calendar
33 Course Catalog
34 Global Learning
35 Learning Goals
36 Research
37 
            Extreme Talent. Radical Collaboration. Results That Matter.
38 Vice President for Research and Innovation
39 Centers & Institutes
40 Technology Licensing (CTL)
41 Graduate Research
42 Undergraduate Research
43 OVPRI Research Publications
44 Public Engagement
45 
            
            CCE helps Buffalo get school food truck rolling
46 Cornell C

In [9]:
cornell_departments = cornell_departments[71:147]

In [11]:
for item in cornell_departments:
    url = item.get('url')
    try:
        item.update({'faculty_page': find_faculty_page(driver, url)})
    except WebDriverException:
        item.update({'faculty_page': None})

In [13]:
with open(sys.path[0] + '/../data/cornell/cornell_directories.json', 'w') as f:
    json.dump(cornell_departments, f)

# UPenn

In [21]:
upenn_departments = []

In [22]:
departments_url = "https://www.sas.upenn.edu/departments/"
no_pages = 1
css_selector = '.field-p-feat-links-links'
upenn_departments_1 = find_department_pages(driver, departments_url, no_pages, css_selector)

In [24]:
tmp = [["Accounting", "https://accounting.wharton.upenn.edu/?_gl=1%2Azsxz4e%2A_ga%2AMTA2Nzk1MjkxNS4xNjUzNjAwNTg2%2A_ga_2QNGY0KQFG%2AMTY1NzEyMTkxMC4yLjEuMTY1NzEyMjI0MC4yNA..%2A_ga_B5B4E387GY%2AMTY1NzEyMTkxMC4xLjEuMTY1NzEyMjI0MC4yNA.."],
 ["Legal Studies and Business Ethics", "https://lgst.wharton.upenn.edu/?_gl=1%2Aa8ns6g%2A_ga%2AMTA2Nzk1MjkxNS4xNjUzNjAwNTg2%2A_ga_2QNGY0KQFG%2AMTY1NzEyMTkxMC4yLjEuMTY1NzEyMjI0MC4yNA..%2A_ga_B5B4E387GY%2AMTY1NzEyMTkxMC4xLjEuMTY1NzEyMjI0MC4yNA.."],
 ["Real Estate", "http://real-estate.wharton.upenn.edu/?_gl=1*jwbi2y*_ga*MTA2Nzk1MjkxNS4xNjUzNjAwNTg2*_ga_2QNGY0KQFG*MTY1NzEyMTkxMC4yLjEuMTY1NzEyMjI0MC4yNA..*_ga_B5B4E387GY*MTY1NzEyMTkxMC4xLjEuMTY1NzEyMjI0MC4yNA.."],
 ["Business Economics and Public Policy", "http://bepp.wharton.upenn.edu/?_gl=1*jwbi2y*_ga*MTA2Nzk1MjkxNS4xNjUzNjAwNTg2*_ga_2QNGY0KQFG*MTY1NzEyMTkxMC4yLjEuMTY1NzEyMjI0MC4yNA..*_ga_B5B4E387GY*MTY1NzEyMTkxMC4xLjEuMTY1NzEyMjI0MC4yNA.."],
 ["Finance", "http://fnce.wharton.upenn.edu/?_gl=1*6qskmv*_ga*MTA2Nzk1MjkxNS4xNjUzNjAwNTg2*_ga_2QNGY0KQFG*MTY1NzEyMTkxMC4yLjEuMTY1NzEyMjI0MC4yNA..*_ga_B5B4E387GY*MTY1NzEyMTkxMC4xLjEuMTY1NzEyMjI0MC4yNA.."],
 ["Health Care Management", "https://hcmg.wharton.upenn.edu/?_gl=1*6qskmv*_ga*MTA2Nzk1MjkxNS4xNjUzNjAwNTg2*_ga_2QNGY0KQFG*MTY1NzEyMTkxMC4yLjEuMTY1NzEyMjI0MC4yNA..*_ga_B5B4E387GY*MTY1NzEyMTkxMC4xLjEuMTY1NzEyMjI0MC4yNA.."],
 ["Management", "http://mgmt.wharton.upenn.edu/?_gl=1*y59h05*_ga*MTA2Nzk1MjkxNS4xNjUzNjAwNTg2*_ga_2QNGY0KQFG*MTY1NzEyMTkxMC4yLjEuMTY1NzEyMjI0MC4yNA..*_ga_B5B4E387GY*MTY1NzEyMTkxMC4xLjEuMTY1NzEyMjI0MC4yNA.."],
 ["Marketing", "http://marketing.wharton.upenn.edu/?_gl=1*y59h05*_ga*MTA2Nzk1MjkxNS4xNjUzNjAwNTg2*_ga_2QNGY0KQFG*MTY1NzEyMTkxMC4yLjEuMTY1NzEyMjI0MC4yNA..*_ga_B5B4E387GY*MTY1NzEyMTkxMC4xLjEuMTY1NzEyMjI0MC4yNA.."],
 ["Operations, Information and Decisions", "https://oid.wharton.upenn.edu/?_gl=1*y59h05*_ga*MTA2Nzk1MjkxNS4xNjUzNjAwNTg2*_ga_2QNGY0KQFG*MTY1NzEyMTkxMC4yLjEuMTY1NzEyMjI0MC4yNA..*_ga_B5B4E387GY*MTY1NzEyMTkxMC4xLjEuMTY1NzEyMjI0MC4yNA.."],
 ["Statistics and Data Science", "http://statistics.wharton.upenn.edu/?_gl=1*r0lezn*_ga*MTA2Nzk1MjkxNS4xNjUzNjAwNTg2*_ga_2QNGY0KQFG*MTY1NzEyMTkxMC4yLjEuMTY1NzEyMjI0MC4yNA..*_ga_B5B4E387GY*MTY1NzEyMTkxMC4xLjEuMTY1NzEyMjI0MC4yNA.."]]

In [27]:
upenn_departments_2 = []
for i in tmp:
    upenn_departments_2.append({'department':i[0], 'url':i[1]})

In [33]:
upenn_departments_3 = [{'department': 'Bioengineering', 'url':'https://be.seas.upenn.edu/'},
                      {'department': 'Chemical and Biomolecular Engineering', 'url':'https://cbe.seas.upenn.edu/'},
                      {'department': 'Computer and Information Science', 'url':'https://www.cis.upenn.edu/'},
                      {'department': 'Electrical and Systems Engineering', 'url':'https://www.ese.upenn.edu/'},
                      {'department': 'Materials Science and Engineering', 'url':'https://mse.seas.upenn.edu/'},
                      {'department': 'Mechanical Engineering and Applied Mechanics', 'url':'https://www.me.upenn.edu/'}]

In [32]:
# Schools without the division of departments
upenn_departments_4 = [{'department': 'Annenberg School for Communication', 'url':'https://www.asc.upenn.edu/'},
                      {'department': 'Stuart Weitzman School of Design', 'url':'https://www.design.upenn.edu/'},
                      {'department': 'Graduate School of Education', 'url':'https://www.gse.upenn.edu/'},
                      {'department': 'Penn Carey Law', 'url': 'https://www.law.upenn.edu/'},
                      {'department': 'School of Social Policy & Practice', 'url': 'https://www.sp2.upenn.edu/'}]

In [34]:
upenn_departments = []
for i in [upenn_departments_1, upenn_departments_2, 
          upenn_departments_3, upenn_departments_4]:
    upenn_departments.extend(i)

In [39]:
for item in upenn_departments:
    url = item.get('url')
    try:
        item.update({'faculty_page': find_faculty_page(driver, url)})
    except WebDriverException:
        item.update({'faculty_page': None})

In [40]:
with open(sys.path[0] + '/../data/upenn/upenn_directories.json', 'w') as f:
    json.dump(upenn_departments, f)

# Harvard

https://en.wikipedia.org/wiki/Outline_of_Harvard_University  
https://www.harvard.edu/academics/schools/

In [41]:
def find_department_pages_harvard(driver):
    driver.get("https://www.fas.harvard.edu/pages/departments-and-areas/")
    items = []
    elements = driver.find_elements(By.TAG_NAME, 'li')
    for element in elements:
        try:
            tmp = element.find_element(By.TAG_NAME, 'a')
            item = {'department': tmp.get_attribute('textContent'), 'url': tmp.get_attribute('href')}
            items.append(item)
        except:
            pass
    return items

In [43]:
harvard_departments_1 = find_department_pages_harvard(driver)

In [47]:
for i in range(len(harvard_departments_1)):
    print(i, harvard_departments_1[i].get('department'))

0 Main Menu
1 Utility Menu
2 Search
3 HARVie
4 Academic Calendar
5 Directory
6 HOME
7 About
8 What is FAS?
9 Dean Claudine Gay
10 Messages from the Dean
11 FAS Leadership
12 Advisory Committees
13 Advancing Racial Justice
14 Support FAS
15 Reports
16 FAS Strategic Planning
17 Core Activities
18 Teaching and Learning
19 Research and Discovery
20 Student Experience
21 Academics & Research
22 Academic Divisions & SEAS
23 Departments and Areas
24 Degree Programs & Courses
25 Centers, Institutes & Societies
26 Libraries
27 Museums
28 Resources
29 Quick Contacts
30 Sexual & Gender-Based Harassment Policy/Resources
31 For Faculty
32 For Staff
33 For Postdocs
34 News
35 African and African American Studies
36 Anthropology
37 Applied Mathematics
38 Applied Physics
39 Art, Film, and Visual Studies
40 Astronomy
41 Bioengineering
42 Celtic Languages and Literatures
43 Chemistry and Chemical Biology
44 Computer Science
45 The Classics
46 Comparative Literature
47 Earth and Planetary Sciences
48 Eas

In [48]:
harvard_departments_1 = harvard_departments_1[35:74]

In [50]:
tmp = [["Architecture", "https://www.gsd.harvard.edu/architecture/"],
       ["Landscape Architecture", "https://www.gsd.harvard.edu/landscape-architecture/"],
       ["Urban Planning and Design", "https://www.gsd.harvard.edu/urban-planning-design/"]]

In [51]:
harvard_departments_2 = []
for i in tmp:
    harvard_departments_2.append({'department':i[0], 'url':i[1]})

In [52]:
departments_url = "https://www.hsph.harvard.edu/departments/"
no_pages = 1
css_selector = '.hsph-bootstrap'
harvard_departments_3 = find_department_pages(driver, departments_url, no_pages, css_selector)

In [54]:
harvard_departments_3 = harvard_departments_3[:9]

In [61]:
for i in harvard_departments_3:
    i.update({'department':i.get('department')[24:]})

In [63]:
harvard_departments_4 = [{'department': 'Harvard Business School', 'url':'https://www.hbs.edu/Pages/default.aspx'},
                         {'department': 'Harvard Divinity School', 'url':'https://hds.harvard.edu/'},
                         {'department': 'Harvard Graduate School of Education', 'url':'https://www.gse.harvard.edu/'},
                         {'department': 'Harvard Kennedy School', 'url':'https://www.hks.harvard.edu/'},
                         {'department': 'Harvard Law School', 'url':'https://hls.harvard.edu/'},
                         {'department': 'T.H. Chan School of Public Health', 'url':'https://hls.harvard.edu/'}]

In [68]:
harvard_departments = []
for i in [harvard_departments_1, harvard_departments_2, 
          harvard_departments_3, harvard_departments_4]:
    harvard_departments.extend(i)

In [70]:
for item in harvard_departments:
    url = item.get('url')
    try:
        item.update({'faculty_page': find_faculty_page(driver, url)})
    except WebDriverException:
        item.update({'faculty_page': None})

In [71]:
with open(sys.path[0] + '/../data/harvard/harvard_directories.json', 'w') as f:
    json.dump(harvard_departments, f)